# Support Vector Regression

In [1]:
# Standard scientific Python imports
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns # for visualisation
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from sklearn.decomposition import PCA
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

import random
random.seed(15)

# Default plotting parameters
font = {'size'   : 18}
plt.rc('font', **font)

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [3]:
# Load training dataset
base_dir = "/Users/Cherry0904/desktop/ArtWorldInsights/ML_modelling/" 
Xy = pd.read_csv(base_dir + 'all_data.csv', squeeze = True)

y = Xy[['logprice']]
X_cts = Xy[['database', 'medium', 'dimensions', 'Followers Per Post (FPP)', 'Instagram performance', 'ArtfactsPresence', 'InsPresence', 'WebsitePresence']]
X = Xy[['database', 'medium', 'dimensions', 'Followers Per Post (FPP)', 'Instagram performance', 'ArtfactsPresence', 'InsPresence', 'WebsitePresence']]
# print(Xy.columns)

# Create instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

# Perform one-hot encoding on the columns of categorical variables 
X_encoder_df1 = pd.DataFrame(encoder.fit_transform(X[['database']]).toarray())
X_encoder_df2 = pd.DataFrame(encoder.fit_transform(X[['medium']]).toarray())
# print(X_encoder_df1.columns)
X_encoder_df1.columns = ['artprice', 'artsper', 'degreeart', 'riseart', 'singulart']
X_encoder_df2.columns = ['drawing', 'painting', 'photo']
# print(X_encoder_df2)

# Merge one-hot encoded columns back with original DataFrame
X_final = X.join(X_encoder_df1)
X_final = X_final.join(X_encoder_df2)
X_final = X_final.drop(['database', 'medium'], axis=1)
# X_final.drop(['medium'], axis=1)
# print(X_final)

# Train-test split
X_tr, X_te, y_tr, y_te = train_test_split(X_final, y, test_size = 0.20 , random_state=15)

# Create version with them together
Xy_tr = pd.concat([X_tr, y_tr], axis = 1)

# Normalise according to training data
scaler = MinMaxScaler()
scaler.fit(X_tr)
X_tr_sc = scaler.transform(X_tr)
X_te_sc = scaler.transform(X_te)

# Futher split to Train-validation sets - 0.8, 0.1, 0.2
# X_train, X_val, y_train, y_val = train_test_split(X_tr, y_tr, test_size = 0.10 , random_state=250)


In [7]:
# Set up the support vector regressor
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf', gamma = 'scale')
# regressor.fit(X_tr_sc, y_tr)

# Grid Search    
# Define grid on hyperparameters
# C controls the regularisation on the error distances
grid = {'C': [0.1, 1, 10, 100], 'kernel': ['rbf', 'poly', 'sigmoid']}
# Gamma is the kernel parameter
# Lower gamma means far-away points are considered for drawing the decision boundary
# grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],'kernel': ['rbf', 'poly', 'sigmoid']}

# Define search - maximise the defined regression metric
search = GridSearchCV(regressor, grid, scoring='neg_mean_squared_error', cv=6, n_jobs=-1)

# Perform the search
results = search.fit(X_tr_sc, y_tr)

# Summarize
print('Negative_mean_squared_error: %.3f' % results.best_score_)      
print('Config: %s' % results.best_params_)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Negative_mean_squared_error: -0.640
Config: {'C': 100, 'kernel': 'rbf'}


In [8]:
# Final evaluation on the held-out test set
regressor = SVR(kernel = 'rbf', C = 100, gamma = 'scale')
regressor.fit(X_tr_sc, y_tr)
pred = regressor.predict(X_te_sc)
rmse = np.sqrt(mean_squared_error(y_te, pred))

print("Test R-square:", regressor.score(X_te_sc, y_te))
print("Test RMSE:", rmse)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Test R-square: 0.4973816801266232
Test RMSE: 0.7952960972056212
